In [1]:
from __future__ import print_function
import cv2
import numpy as np
from skimage.metrics import structural_similarity
import matplotlib.pyplot as plt
from cv2 import cv2 as cv2
from PIL import Image
import imagehash
from moviepy.editor import VideoFileClip
import os
from collections import deque
import time
import json

## parameter

In [2]:
gap_factor=20 ## num of detection within each second
match_ratio=0.65 ##the distance ratio thershold between best match and second match
good_match_rate=1 ##the rate of matches to be chosen in all
ransacReprojThreshold=6
min_match_count=10


## notation

for keypoint, attribute to use are:
    pt 
    angle
    response
    size
    octave
    class_id：object
for match, attribute to use are:
    distance
    queryIdx
    trainIdx
    imgIdx
    
    print(kp1[i.queryIdx].pt) location before move
    print(kp2[i.trainIdx].pt) location after move

## test begin

In [3]:
def main():
    out_sequence=[]
    ##load test video
    video=os.path.abspath(os.curdir)+'/demo.mp4'
    videoCapture = cv2.VideoCapture(video)
    
    ##detect for gap_factor #gap_dected frames
    fps = videoCapture.get(cv2.CAP_PROP_FPS)
    print(fps)
    gap_dected=int(fps//gap_factor)
    
    ##load first frame
    success, frame_old = videoCapture.read()
    i,j=0,0
    _SaveImage(frame_old[80:frame_old.shape[0]-40,:],os.path.abspath(os.curdir)+'/test',j)
    print('save image:',i)
    
#     predict=deque([],maxlen=gap_factor)
#     for _ in range(gap_factor):
#         predict.append(None)
    
    while success:
        i+=1
    ##finish process if all frames loaded
        success, frame_new = videoCapture.read()
        if success==False:
            print('vedio detection finish')
#           sys.exit() 
            return out_sequence
        if (i % gap_dected == 0):
            j+=1
            _SaveImage(frame_new[80:frame_new.shape[0]-40,:],os.path.abspath(os.curdir)+'/test',j)
            print('save image:',i)
            
    ##predict 
            if _AlignImages(frame_old,frame_new,i) != None:
                predict=_AlignImages(frame_old,frame_new,i)
               
                print(predict)
                
                res=_Predict(predict)
                out_sequence.append(res)
                print(i//30,'s',i%30,'frame',res)
            
            
        frame_old=frame_new
    
            
        
        

        
        
    
    

In [4]:
def _SaveImage(image,addr,num):
    address = addr +'/'+ str(num)+ '.jpg'
    cv2.imwrite(address,image)

def _AlignImages(img1,img2,filenum):
    # initiate ORB detector
    orb = cv2.ORB_create(5000)
    
    ##check bilateral banner, eliminate stick part
#     up_idx,down_idx=_SplitImage(img1,img2)
#     img1 = img1[up_idx:down_idx,:]
#     img2= img2[up_dx:down_idx,:]
    img1 = img1[80:img1.shape[0]-40,:]
    img2 = img2[80:img2.shape[0]-40,:]
    img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    count=0
    
    for i in img2.sum(axis=1):
        if i==0:
            count+=1
    if count/img2.sum(axis=1).shape[0]>0.2:
        return 'open','picture'
    
    ##find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)
    if des2 is None: 
        return 'click','in'
    
    ##match with Brute-Force
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    ##find best 2 matches for each kp
    matches = bf.knnMatch(des1, trainDescriptors = des2, k = 2)
#     print(len(matches))
    ##ratio test
#     debug=[e for e in matches ]
#     with open('debug.txt','a') as file0:
#         print(debug,file=file0)
#         print('\n',file=file0)
    
    if len(matches)==0 or len(matches[0])==1 :
        return 'click','in'
    
    else:
        good = [m for (m,n) in matches if m.distance < match_ratio*n.distance]
    ##sort and choose the best part of matches
#     good = sorted(good, key = lambda x:x.distance)
#     good = good[:int(len(good) * good_match_rate)]
    img3 = cv2.drawMatches(img1,kp1,img2,kp2,good[int(0.75*len(good)):int(1*len(good))],None,flags=2)
    _SaveImage(img3,os.path.abspath(os.curdir)+'/compare',filenum)
    
    if len(good)>min_match_count:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good[int(0.75*len(good)):int(1*len(good))] ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good[int(0.75*len(good)):int(1*len(good))]]).reshape(-1,1,2)
        
        src_pts,dst_pts=_Recheck(src_pts,dst_pts,img1.shape)
        if dst_pts.shape[0]<=50:
            return 'click','in'
        res1,res2=_Good_inter_check(src_pts,dst_pts)
        if res1!=0:
            return res1,res2
        ##find (3x3) transform matrix
        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,ransacReprojThreshold)
        t_x = M[0][2]
        t_y = M[1][2]
        return (t_x,t_y)
    else:
        return None
        

def _Recheck(src_pts,dst_pts,fig_shape):
    s=[]
    d=[]
    for i in range(src_pts.shape[0]):
        sc=src_pts[i]
        ds=dst_pts[i]
        dx=abs(sc[0][0]-ds[0][0])
        dy=abs(sc[0][1]-ds[0][1])
        if dx/fig_shape[1]<0.3 and dy/fig_shape[0]<0.3:
            s.append(sc)
            d.append(ds)
    s=np.array(s)
    d=np.array(d)
#     print(s.shape)
    return s,d
    
        
def _Good_inter_check(src_pts,dst_pts):
    res=[]
#     res2=[]
    for i in range(src_pts.shape[0]):
        s=src_pts[i]
        d=dst_pts[i]
        dy=abs(s[0][1]-d[0][1])
#         dx=abs(s[0][0]-d[0][0])
        res.append(dy)
#         res2.append(dx)
    res.sort()
#     res2.sort()
    allave=np.mean(res)
    lastave=np.mean(res[9*int(len(res)/10):-1])
#     allave2=np.mean(res2)
#     lastave2=np.mean(res2[9*int(len(res2)/10):-1])
    if (lastave>=5*allave and allave>=2):
        return 'buy good','change'
#     if (lastave2>=5*allave2 and allave2>=2):
#         return 'good picture','change'
    else:
        return 0,0
    
        
        

def _Predict(predict):
    
    res= __Pattern(predict[0],predict[1])
    return res
    
def __Pattern(t_x,t_y):
    if type(t_x)!=str and type(t_y)!=str:
        res=[]
        if abs(t_x)<1 and abs(t_y)<1:
            return 'stop'
        if abs(t_y/t_x)>3:
            if (t_y>3):
                res.append('down')
            if (t_y<-3):
                res.append('up')
        
        if abs(t_x/t_y)>3:
            if (t_x>3):
                res.append('right')
            if(t_x<-3):
                res.append('left')
        if len(res)==0:
            return 'stop'
        else:
            major={'up':0,'down':0,'right':0,'left':0,'stop':0}
            for e in res:
                major[e]+=1
            for k, v in major.items():
                if v == max(major.values()):
                    return k        
        
    else:
#         if t_x=='click' and t_y=='in':
        return t_x+t_y
        
        
        
        
        
#         if abs(t_x)<0.1 and abs(t_y)<0.1:
#     #         print('stop')
#             return 'stop'
    
#         elif abs(t_y/t_x)>2:
#     #         print('up,down')
#             if (t_y>5):
#     #             print('down')
#                 return 'down'
# #             elif (t_y<-0.5):
# #     #             print('up')
# #                 return 'up'
#             else:
#                 return 'up'
            
#         elif abs(t_x/t_y)>5:
#             if (t_x>3):
#     #             print('down')
#                 return 'right'
#             elif(t_x<-3):
#                 return 'left'
#             else:
#     #             print('up')
#                 return 'stop'


#         else:
#             return 'stop'
   
        
        
          
    
    
    
    
    
    
    
    

In [5]:
# major={'stop':1,'up':0,'right':0,'left':0}

In [6]:
out_sequence=main()

30.0
save image: 0
save image: 1
(-4.7986854696923505e-15, 2.2675536214946323e-15)
0 s 1 frame stop
save image: 2
(0.006970170292136587, 0.009789390370122058)
0 s 2 frame stop
save image: 3
(0.00046617301482127676, -0.009554749986617926)
0 s 3 frame stop
save image: 4
(-1.6992303659146484e-13, -3.4603991698178296e-14)
0 s 4 frame stop
save image: 5
(-2.384516104199004e-13, -6.588788889662146e-14)
0 s 5 frame stop
save image: 6
(-0.002469850652402011, 0.006323450264693732)
0 s 6 frame stop
save image: 7
(0.005667216264992179, -0.01941726942105719)
0 s 7 frame stop
save image: 8
(0.0027339420540350872, 0.010013315134830206)
0 s 8 frame stop
save image: 9
(-0.03517995934158645, -0.0562246823736888)
0 s 9 frame stop
save image: 10
(-2.7744442091062943e-14, 2.0516036182732713e-14)
0 s 10 frame stop
save image: 11
(-0.03793899553553168, -0.055200005373589454)
0 s 11 frame stop
save image: 12
(-2.906930770886172e-13, -3.645134057274051e-13)
0 s 12 frame stop
save image: 13
(-0.007033024913494

In [7]:
out_sequence

['stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'up',
 'up',
 'up',
 'up',
 'up',
 'up',
 'stop',
 'up',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'stop',
 'up',
 'up',
 'up',
 

In [11]:
def logic(out_sequence):
    my_dict={'stop':0,'up':1,'down':2,'left':3,'right':4,'clickin':5,'open picture':6,'buy good change':7}
    left_idx_list=[]
    res=[]
    click_symbol=0
    click_add=0
    for i in range(len(out_sequence)):
        if out_sequence[i]=='stop':
            click_add=0
            if click_symbol==1:
                left_idx_list=[]
                click_symbol=0
            elif click_symbol==0:
                if len(left_idx_list)!=0:
                    for i in range(len(left_idx_list)):
                        res.append(my_dict['open picture'])
                    left_idx_list=[]
            res.append(my_dict['stop'])
        if out_sequence[i]=='left':
            click_add=0
            left_idx_list.append(i)
        if out_sequence[i]=='clickin':
            click_symbol=1
            if click_add==0:
                res.append(my_dict['clickin'])
                click_add=1
        if out_sequence[i]=='right' or out_sequence[i]=='up' or out_sequence[i]=='down' or out_sequence[i]=='buy good change':
            res.append(out_sequence[i])
    return res
            
            
        
                
                
            
            
            
        
        
        
    

In [12]:
logic(['stop','stop','stop','left','left','clickin','clickin','stop','stop','left','left','left','stop','stop'])

[0, 0, 0, 5, 0, 0, 6, 6, 6, 0, 0]

In [8]:
a.append(1)
a.append(None)

In [13]:
a


deque([1, None])

In [20]:
test()

'stop'

In [82]:
a=1

In [83]:
type(a)

int

In [84]:
type(str(a))

str

In [85]:
type(a)!=str

True